# 앙상블 예제

In [2]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier,VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
import pandas as pd

# 필요한 라이브러리를 호출
from sklearn.model_selection import train_test_split
from sklearn.compose        import make_column_transformer
from sklearn.impute import KNNImputer
from imblearn.pipeline      import pipeline
from imblearn.pipeline      import make_pipeline
from sklearn.impute         import SimpleImputer, KNNImputer
from sklearn.preprocessing  import MinMaxScaler, OneHotEncoder
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import ADASYN
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV    # 교차검증
from sklearn.metrics   import classification_report     # 평가
import pickle
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor # Bagging Model 
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv('final_3.0.csv')
df.columns

C:\Users\DMC CONET\AppData\Local\Temp\ipykernel_23008\1763953460.py:1: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('final_3.0.csv')


Index(['Unnamed: 0', '사업자등록번호', '데이터수집일자', '조사년도', '재무보고서구분코드', '재무보고서명칭',
       '재무제표기준년월', '재무제표계정과목코드', '재무제표계정과목명', '재무제표계정과목금액', '구성비율', '증감율',
       '기업코드', '기업명', '업종', '업종명', '기업규모', '기업규모명', 'CRI등급', 'CRI기준일', '결산일자',
       '사업자등록번호_마스킹', '대표자명', '기업영문명', '종업원수', '종업원수기준년월', '산업분류코드',
       '산업분류코드설명', '기업형태', '설립일', '이메일주소', '전화번호', '팩스전화번호', '우편번호', '주소',
       '상세주소', '매출액', '영업이익', '자본총계금액', '부채총계금액', '자산총계금액', '산업분류코드설명중분류',
       '산업분류코드설명소분류', '중분류코드', '중분류항목명', '소분류코드', '소분류항목명'],
      dtype='object')

In [4]:
data = df[[ '부채총계금액', '자산총계금액', 'CRI등급']].dropna()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 411158 entries, 0 to 469089
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   부채총계금액  411158 non-null  float64
 1   자산총계금액  411158 non-null  float64
 2   CRI등급   411158 non-null  object 
dtypes: float64(2), object(1)
memory usage: 12.5+ MB


In [5]:
X =  data[[ '부채총계금액', '자산총계금액']]
Y = data['CRI등급']
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=2021,shuffle=True)

# 약한 학습기 구축
log_model = LogisticRegression()
rnd_model = RandomForestClassifier()
xgb_model = XGBClassifier
svm_model = SVC()

# 앙상블 모델 구축
# 만약에 모든 모델이 predict_proba() 메서드가 있으면, 예측의 평균을 내어 soft voting(간접 투표)도 할수 있다.
# 간접 투표 방식은 확률이 높은 투표에 비중을 두기 때문에 성능이 더 높다. (voting='soft' 사용)
# svc는 기본적으로 predict_proba를 제공하지 않아, probability = True 지정 해야 사용 가능
# 대신 svc에서 probability = True를 지정하면 교차 검증을 사용해서 확률을 추정하기 때문에 훈련 속도 느려짐
# 대신 성능을 올라감

voting_model = VotingClassifier(
    estimators=[('lr',log_model),('rf',rnd_model),('svc',svm_model)], # 3개의 약한 학습기
    voting='hard' # 직접 투표(hard voting)
)

# 앙상블 모델 학습
voting_model.fit(x_train,y_train)

# 모델 비교
for model in (log_model,rnd_model,svm_model,voting_model):
    model.fit(x_train,y_train)
    y_pred = model.predict(x_test)
    print(model.__class__.__name__," : ",accuracy_score(y_test,y_pred))


C:\Users\DMC CONET\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\DMC CONET\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/lin

LogisticRegression  :  0.02087589583941369
RandomForestClassifier  :  1.0
SVC  :  0.246400428057204


C:\Users\DMC CONET\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


VotingClassifier  :  0.2705272886467555


# pipeline ensamble

- 출처 : https://dnai-deny.tistory.com/27

In [ ]:
from scipy.special import comb
import math

def ensemble_error(n_classifier, error):
    k_start = int(math.ceil(n_classifier / 2.))
    probs = [comb(n_classifier, k) * error**k * (1-error)**(n_classifier - k)
             for k in range(k_start, n_classifier + 1)]
    return sum(probs)

ensemble_error(n_classifier=11, error=0.25)

In [ ]:
import numpy as np

error_range = np.arange(0.0, 1.01, 0.01)
ens_errors = [ensemble_error(n_classifier=11, error=error)
              for error in error_range]

import matplotlib.pyplot as plt


plt.plot(error_range, 
         ens_errors, 
         label='Ensemble error', 
         linewidth=2)

plt.plot(error_range, 
         error_range, 
         linestyle='--',
         label='Base error',
         linewidth=2)

plt.xlabel('Base error')
plt.ylabel('Base/Ensemble error')
plt.legend(loc='upper left')
plt.grid(alpha=0.5)
plt.show()

In [ ]:
from sklearn.base import BaseEstimator
from sklearn.base import ClassifierMixin
from sklearn.preprocessing import LabelEncoder
from sklearn.externals import six
from sklearn.base import clone
from sklearn.pipeline import _name_estimators
import numpy as np
import operator
class MajorityVoteClassifier(BaseEstimator, ClassifierMixin):

    def __init__(self, classifiers,
        vote='classlabel', weights=None):

         #vote의 기본값은 classlabel. classlabel이면 다수인 클래스 레이블 인덱스, 'probability'이면 확률 합이 가장 큰 인덱스로 예측
         #weight는 배열타입. 기본값 None
         #Classifier은 앙상블에 사용할 분류기

        self.classifiers = classifiers 
        self.named_classifiers = {key: value for key, value in _name_estimators(classifiers)}
        self.vote = vote
        self.weights = weights

    def fit(self, X, y):

         #X = 훈련 샘플 행렬. 
         #y = 

        self.lablenc_ = LabelEncoder()
        self.lablenc_.fit(y)
        self.classes_ = self.lablenc_.classes_
        self.classifiers_ = []
         for clf in self.classifiers:
            fitted_clf = clone(clf).fit(X, self.lablenc_.transform(y))
            self.classifiers_.append(fitted_clf)
        return self

In [ ]:
def predict(self, X):
""" 
X : 샘플 데이터 행렬
maj_vote : 예측된 클래스 레이블
"""
    if self.vote == 'probability':
        maj_vote = np.argmax(self.predict_proba(X),axis=1)
    else: # 'classlabel' 투표 
        # clf.predict 로 결과를 모읍니다.
        predictions = np.asarray([clf.predict(X) for clf in self.classifiers_]).T
        maj_vote = np.apply_along_axis(
        lambda x:
        np.argmax(np.bincount(x, weights=self.weights)),
        axis=1,
        arr=predictions)
        maj_vote = self.lablenc_.inverse_transform(maj_vote)
    return maj_vote

def predict_proba(self, X):
""" 
X : 샘플 데이터 행렬
avg_proba : 샘플마다 가중치가 적용된 클래스의 평균 확률
"""
    probas = np.asarray([clf.predict_proba(X)
                         for clf in self.classifiers_])
    avg_proba = np.average(probas, axis=0, weights=self.weights)
    return avg_proba

def get_params(self, deep=True):
""" GridSearch를 위해 분류기 매개변수 이름을 반환 """
    if not deep:
    return super(MajorityVoteClassifier,
                 self).get_params(deep=False)
    else:
        out = self.named_classifiers.copy()
        for name, step in\
        six.iteritems(self.named_classifiers):
    for key, value in six.iteritems(
            step.get_params(deep=True)):
        out['%s__%s' % (name, key)] = value
return out

In [9]:
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

In [ ]:


iris = datasets.load_iris()
X, y = iris.data[50:, [1, 2]], iris.target[50:]
le = LabelEncoder()
y = le.fit_transform(y)

# X_train, X_test, y_train, y_test =\
#        train_test_split(X, y, 
#                         test_size=0.5, 
#                         random_state=1,
#                         stratify=y)

In [ ]:
X = data[[ '부채총계금액', '자산총계금액']]
Y = data['CRI등급']
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.3,random_state=2021,shuffle=True)

In [ ]:


clf1 = LogisticRegression(solver='liblinear',
                          penalty='l2', 
                          C=0.001,
                          random_state=1)

clf2 = DecisionTreeClassifier(max_depth=1,
                              criterion='entropy',
                              random_state=0)

clf3 = KNeighborsClassifier(n_neighbors=1,
                            p=2,
                            metric='minkowski')

pipe1 = Pipeline([['sc', StandardScaler()],
                  ['clf', clf1]])
pipe3 = Pipeline([['sc', StandardScaler()],
                  ['clf', clf3]])

clf_labels = ['Logistic regression', 'Decision tree', 'KNN']

print('10-겹 교차 검증:\n')

for clf, label in zip([pipe1, clf2, pipe3], clf_labels):
    scores = cross_val_score(estimator=clf,
                             X=X_train,
                             y=y_train,
                             cv=10,
                             scoring='roc_auc')
    print("ROC AUC: %0.2f (+/- %0.2f) [%s]"
          % (scores.mean(), scores.std(), label))

In [2]:
input_ff = pd.read_csv('전처리완료한데이터.csv', encoding='cp949')

- 구할 수 있는 값
    1) 총자산증가율 (자산총계, 부채와자본총계)
    2) 매출액증가율 (매출 증가율)
    
    3) 유동비율 (유동비율: 유동자산, 유동부채)
    4) 자기자본비율 (자기자본 / 부채와자본총계)
    
    5) 부채비율 (유동부채+비유동부채) / (자기자본) -> 불가능
    
    6) 이자보상비율 (영업이익/이자비용) * 100
    7) 적립금비율 ((자본잉여금+이익잉여금)/자기자본)*100
    8) 총자본순이익율 (당기순이익('손실') / 자산총계) * 100
    9) 매출영업이익율 (영업이익/매출액) * 100
    
    10) 총자산회전율 (매출액/자산총계)
    11) 매출채권회전율 (매출액/매출채권(평균))
    
    12) 당좌비율 (당좌자산/유동부채)* 100
    
> 필요한 컬럼명: '자산총계', '부채와자본총계', '매출 증가율', '유동자산', '유동부채', '자기자본', '영업이익', '이자비용', '자본잉여금', '이익잉여금', '당기순이익('손실')', '매출액', '총자산',
'매출채권(평균)', '당좌자산'

> 사용가능한 컬럼명: '자산총계', '부채와자본총계', '유동자산', '유동부채', '영업이익', '이자비용', '자본잉여금', '이익잉여금', '당기순이익('손실')', '매출액', 
'매출채권(평균)', '당좌자산'

drop해야하는 재무제표계정과목명: '유동비율' (제거), '자기자본' (제거), '총자산' (제거), '매출 증가율' (제거), '유형자산 증가율' (제거), '총자산 증가율' (제거), '차입금' (제거), '회사채' (제거)

# 중분류

In [ ]:
'''
class_list = input_ff['중분류항목명'].value_counts().index.tolist()[:3] # 제일많은 빈도수의 중분류
class_list


cond1 = (input_ff['중분류항목명'] == class_list[0])  # '전문직별 공사업'
cond2 = (input_ff['중분류항목명'] == class_list[1]) # '컴퓨터 프로그래밍, 시스템 통합 및 관리업'
cond3 = (input_ff['중분류항목명'] == class_list[2]) # '출판업'

cl1 = input_ff.loc[cond1] ##  '컴퓨터 프로그래밍, 시스템 통합 및 관리업'
cl2 = input_ff.loc[cond1]# 전문직별 공사업
cl3 = input_ff.loc[cond1] # '출판업'
# 중분류에대한 모델 , 분석기법 동일한지. 나중 = input 넣어서 output
# 변수정의서, 재무계산식 변수정의서 
# macro avg의 대한결과로 보고 
'''

In [96]:
input_ff_cl.columns

Index(['사업자등록번호', '중분류항목명', '당좌자산', '매출액', '부채와자본총계', '영업이익', '유동부채', '유동자산',
       '이익잉여금', '이자비용', '자본잉여금', '자산총계'],
      dtype='object', name='재무제표계정과목명')

In [94]:
input_ff_cl = input_ff.pivot_table(index=['사업자등록번호','중분류항목명'], columns=['재무제표계정과목명'], values='재무제표계정과목금액')
input_ff_cl = input_ff_cl.reset_index() 
중분류별_col = input_ff_cl.merge(input_ff, right_on='사업자등록번호', left_on='사업자등록번호', how='left')[['중분류항목명_x', '당좌자산', '매출액', '부채와자본총계', '영업이익', '유동부채', '유동자산',
       '이익잉여금', '이자비용', '자본잉여금', '자산총계', '조사년도', '재무제표기준년월',  '기업규모명', 'CRI등급',  '종업원수', ]]
중분류별_col.head()

,중분류항목명_x,당좌자산,매출액,부채와자본총계,영업이익,유동부채,유동자산,이익잉여금,이자비용,자본잉여금,자산총계,조사년도,재무제표기준년월,기업규모명,CRI등급,종업원수
0,전문직별 공사업,2123184.0,4897308.8,NaN,205107.8,596463.0,3393324.2,2244370.6,27917.6,NaN,4565549.6,2021,202112,중소기업,B2,31.0
1,전문직별 공사업,2123184.0,4897308.8,NaN,205107.8,596463.0,3393324.2,2244370.6,27917.6,NaN,4565549.6,2021,202112,중소기업,B2,31.0
2,전문직별 공사업,2123184.0,4897308.8,NaN,205107.8,596463.0,3393324.2,2244370.6,27917.6,NaN,4565549.6,2018,201812,중소기업,B2,31.0
3,전문직별 공사업,2123184.0,4897308.8,NaN,205107.8,596463.0,3393324.2,2244370.6,27917.6,NaN,4565549.6,2018,201812,중소기업,B2,31.0
4,전문직별 공사업,2123184.0,4897308.8,NaN,205107.8,596463.0,3393324.2,2244370.6,27917.6,NaN,4565549.6,2018,201812,중소기업,B2,31.0


In [71]:
# 재무제표 계산식을 이용한 새로운 변수 설정

# 매출액/총자산(=자산총계)

# 총자산증가율	"(당기말총자산/전기말총자산)*100-100 = (당기말 자산총계/전기말 자산총계)*100-100
# 자산총계 =총자본 = 총자산 =부채총계+자본총계(자기자본)

# 중분류별_col['총자산회전율'] = 중분류별_col['매출액'] - 중분류별_col['자본총계']

# 당기매출액 : 기업이 일정 기간 동안 얻은 모든 수익에서 지출한 모든 비용을 공제하고 순수하게 이익으로 남은 몫
# 매출액증가율 : 전년에 비해 얼마나 매출액이 증가했는지를 보여주는 성장성 지표
# 매출액증가율	(당기매출액/전기매출액)*100-100
# 중분류별_col['매출액증가율'] = 

# 유동비율	(유동자산/유동부채)*100
중분류별_col['유동비율'] = (중분류별_col['유동자산'] / 중분류별_col['유동부채']) * 100 #

# 차입금의존도	((차입금+회사채+유동성장기부채)/자본총계)*100
# 중분류별_col['차입금의존도'] =  

# 자기자본비율	"(자기자본(=자본총계)/총자본(=자산총계))*100
# 중분류별_col['자기자본비율'] = (중분류별_col['자본총계'] / 중분류별_col['자산총계']) * 100


# 부채비율	((유동부채+비유동부채)/자본총계(=자기자본))*100
# 중분류별_col['부채비율'] = ((중분류별_col['유동부채'] + 중분류별_col['비유동부채']) / 중분류별_col['자본총계']) * 100

# 이자보상비율	(영업이익/이자비용)*100
중분류별_col['이자보상비율'] = (중분류별_col['영업이익'] / 중분류별_col['이자비용']) * 100

# 적립금비율	((자본잉여금+이익잉여금)/자본총계(=자기자본))*100
# 중분류별_col['적립금비율'] = ((중분류별_col['자본잉여금'] + 중분류별_col['이익잉여금']) / 중분류별_col['자본총계']) * 100

# 총자본순이익율	(당기순이익(손실)/총자본(=자산총계))*100
# 중분류별_col['총자본순이익율'] = 중분류별_col['당기순이익(손실)'] / 중분류별_col['자산총계']#

# 매출영업이익율	(영업이익/매출액)*100
중분류별_col['매출영업이익률'] = round((중분류별_col['영업이익'] / 중분류별_col['매출액']) * 100 , 2)

# 총자산회전율	매출액/총자산(=자산총계)
중분류별_col['총자산회전율'] = 중분류별_col['매출액'] / 중분류별_col['자산총계']#

# 매출채권회전율	매출액/매출채권(평균)
# 중분류별_col['매출채권회전율'] = 중분류별_col['매출액'] / 중분류별_col['매출채권'].mean()#

# 유동비율	(유동자산/유동부채)*100
중분류별_col['유동비율'] = (중분류별_col['유동자산'] / 중분류별_col['유동부채'])# 

# 당좌비율	(당좌자산/유동부채)*100
중분류별_col['당좌비율'] = (중분류별_col['당좌자산'] / 중분류별_col['유동부채']) * 100

중분류별_col.columns

Index(['중분류항목명_x', '당좌자산', '매출액', '부채와자본총계', '영업이익', '유동부채', '유동자산', '이익잉여금',
       '이자비용', '자본잉여금', '자산총계', '조사년도', '재무제표기준년월', '기업규모명', 'CRI등급', '종업원수',
       '유동비율', '이자보상비율', '매출영업이익률', '총자산회전율', '당좌비율'],
      dtype='object')

In [5]:
중분류별_col.isnull().sum()

중분류항목명_x        0
당좌자산           20
매출액            14
부채와자본총계     82251
영업이익            0
유동부채           27
유동자산            0
이익잉여금          20
이자비용         4640
자본잉여금       62012
자산총계            0
조사년도            0
재무제표기준년월        0
기업규모명           0
CRI등급           0
종업원수            0
유동비율           27
이자보상비율       4640
매출영업이익률        14
총자산회전율         14
당좌비율           47
dtype: int64

In [92]:
중분류별_col['중분류항목명_x'].value_counts()

전문직별 공사업                   30824
컴퓨터 프로그래밍, 시스템 통합 및 관리업    26135
출판업                        25328
Name: 중분류항목명_x, dtype: int64

In [73]:
import numpy as np
# ff = ff.replace([np.inf, -np.inf], 1)

import numpy as np

중분류별_col = 중분류별_col.replace([np.inf, -np.inf], 1)


X = 중분류별_col[[ '당좌자산', '매출액', #'부채와자본총계', '영업이익', '유동부채', '유동자산', '이익잉여금','자본잉여금','이자비용',  '자산총계',
       '조사년도', '종업원수', '재무제표기준년월', '중분류항목명_x', '기업규모명',
       '유동비율', '이자보상비율', '매출영업이익률', '총자산회전율', '당좌비율']] 
Y = 중분류별_col['CRI등급']

In [74]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, random_state=1234, test_size=0.4, shuffle=True, stratify=Y)

In [75]:
numeric_pipe = make_pipeline(SimpleImputer(), MinMaxScaler())
# category_pipe = make_pipeline(SimpleImputer(strategy='most_frequent'), OrdinalEncoder())

numeric_list = X.describe().columns.tolist()
# category_list = X.describe(include='object').columns.tolist()

preprocessing_pipe = make_column_transformer((numeric_pipe, numeric_list), 
#                                              (category_pipe, category_list)
                                            )

# 중분류 model

# DecisionTreeClassifier

In [97]:
dt_model = dict({})
for i in range(5, 50):
    model_pipe = make_pipeline(preprocessing_pipe, DecisionTreeClassifier(max_depth=i,
                                  criterion='entropy',
                                  random_state=0))


    grid_model = GridSearchCV(model_pipe, cv=5, param_grid={}, n_jobs=-1)
    grid_model.fit(X_train, Y_train)

    # 6. 평가
    best_model = grid_model.best_estimator_

    Y_train_pred = best_model.predict(X_train)
    Y_test_pred = best_model.predict(X_test)

    # 평가

    print(f'depth 값 : {i}')
    print(classification_report(Y_train, Y_train_pred))

    print(classification_report(Y_test,Y_test_pred))
    dt_model[f'{i}_depth'] = accuracy_score(Y_test,Y_test_pred)
dt_model

depth 값 : 5


C:\Users\DMC CONET\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DMC CONET\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DMC CONET\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

              precision    recall  f1-score   support

          A1       0.59      0.15      0.24       496
          A2       0.69      0.15      0.25      2555
          A3       0.51      0.17      0.26      6260
          B1       0.36      0.22      0.27      8240
          B2       0.36      0.73      0.48     11600
          B3       0.32      0.41      0.36      7163
          C1       0.42      0.10      0.16      4456
          C2       0.41      0.13      0.19      4298
          C3       0.44      0.65      0.52      4293
           D       0.00      0.00      0.00        11

    accuracy                           0.38     49372
   macro avg       0.41      0.27      0.27     49372
weighted avg       0.41      0.38      0.34     49372



C:\Users\DMC CONET\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DMC CONET\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DMC CONET\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

              precision    recall  f1-score   support

          A1       0.53      0.10      0.17       331
          A2       0.68      0.16      0.25      1703
          A3       0.48      0.16      0.24      4173
          B1       0.36      0.21      0.27      5494
          B2       0.36      0.74      0.48      7734
          B3       0.31      0.39      0.35      4776
          C1       0.44      0.10      0.16      2971
          C2       0.39      0.12      0.19      2865
          C3       0.44      0.65      0.52      2861
           D       0.00      0.00      0.00         7

    accuracy                           0.37     32915
   macro avg       0.40      0.26      0.26     32915
weighted avg       0.40      0.37      0.33     32915

depth 값 : 6


...

              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       331
          A2       1.00      1.00      1.00      1703
          A3       1.00      1.00      1.00      4173
          B1       1.00      1.00      1.00      5494
          B2       1.00      1.00      1.00      7734
          B3       1.00      1.00      1.00      4776
          C1       1.00      1.00      1.00      2971
          C2       1.00      1.00      1.00      2865
          C3       1.00      1.00      1.00      2861
           D       1.00      1.00      1.00         7

    accuracy                           1.00     32915
   macro avg       1.00      1.00      1.00     32915
weighted avg       1.00      1.00      1.00     32915

depth 값 : 33
              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       496
          A2       1.00      1.00      1.00      2555
          A3       1.00      1.00      1.00      6260
          B

depth 값 : 38
              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       496
          A2       1.00      1.00      1.00      2555
          A3       1.00      1.00      1.00      6260
          B1       1.00      1.00      1.00      8240
          B2       1.00      1.00      1.00     11600
          B3       1.00      1.00      1.00      7163
          C1       1.00      1.00      1.00      4456
          C2       1.00      1.00      1.00      4298
          C3       1.00      1.00      1.00      4293
           D       1.00      1.00      1.00        11

    accuracy                           1.00     49372
   macro avg       1.00      1.00      1.00     49372
weighted avg       1.00      1.00      1.00     49372

              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       331
          A2       1.00      1.00      1.00      1703
          A3       1.00      1.00      1.00      4173
          B

              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       331
          A2       1.00      1.00      1.00      1703
          A3       1.00      1.00      1.00      4173
          B1       1.00      1.00      1.00      5494
          B2       1.00      1.00      1.00      7734
          B3       1.00      1.00      1.00      4776
          C1       1.00      1.00      1.00      2971
          C2       1.00      1.00      1.00      2865
          C3       1.00      1.00      1.00      2861
           D       1.00      1.00      1.00         7

    accuracy                           1.00     32915
   macro avg       1.00      1.00      1.00     32915
weighted avg       1.00      1.00      1.00     32915

depth 값 : 44
              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       496
          A2       1.00      1.00      1.00      2555
          A3       1.00      1.00      1.00      6260
          B

depth 값 : 49
              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       496
          A2       1.00      1.00      1.00      2555
          A3       1.00      1.00      1.00      6260
          B1       1.00      1.00      1.00      8240
          B2       1.00      1.00      1.00     11600
          B3       1.00      1.00      1.00      7163
          C1       1.00      1.00      1.00      4456
          C2       1.00      1.00      1.00      4298
          C3       1.00      1.00      1.00      4293
           D       1.00      1.00      1.00        11

    accuracy                           1.00     49372
   macro avg       1.00      1.00      1.00     49372
weighted avg       1.00      1.00      1.00     49372

              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       331
          A2       1.00      1.00      1.00      1703
          A3       1.00      1.00      1.00      4173
          B

{'5_depth': 0.37125930426857057,
 '6_depth': 0.3973568281938326,
 '7_depth': 0.4456023089776698,
 '8_depth': 0.5192465441288167,
 '9_depth': 0.5989366550205074,
 '10_depth': 0.6688136108157375,
 '11_depth': 0.748564484277685,
 '12_depth': 0.8215099498708796,
 '13_depth': 0.8830320522558104,
 '14_depth': 0.9318547774570864,
 '15_depth': 0.9624791128664743,
 '16_depth': 0.98164970378247,
 '17_depth': 0.9943186996809965,
 '18_depth': 0.9995442807230746,
 '19_depth': 1.0,
 '20_depth': 1.0,
 '21_depth': 1.0,
 '22_depth': 1.0,
 '23_depth': 1.0,
 '24_depth': 1.0,
 '25_depth': 1.0,
 '26_depth': 1.0,
 '27_depth': 1.0,
 '28_depth': 1.0,
 '29_depth': 1.0,
 '30_depth': 1.0,
 '31_depth': 1.0,
 '32_depth': 1.0,
 '33_depth': 1.0,
 '34_depth': 1.0,
 '35_depth': 1.0,
 '36_depth': 1.0,
 '37_depth': 1.0,
 '38_depth': 1.0,
 '39_depth': 1.0,
 '40_depth': 1.0,
 '41_depth': 1.0,
 '42_depth': 1.0,
 '43_depth': 1.0,
 '44_depth': 1.0,
 '45_depth': 1.0,
 '46_depth': 1.0,
 '47_depth': 1.0,
 '48_depth': 1.0,
 '49_

# RandomForestClassifier

In [98]:
# drict practice
model = dict({})
model = {1: ['56', '85'], 2:48}
pd.DataFrame(model)

,1,2
0,56,48
1,85,48


In [99]:
# model_list = ['RandomForestClassifier', 'DecisionTreeClassifier', 'KNeighborsClassifier']
rf_model = dict({})
# for model in model_list:
for i in range(5, 50):
    model_pipe = make_pipeline(preprocessing_pipe, RandomForestClassifier(max_depth=i,
                                  criterion='entropy',
                                  random_state=0))


    grid_model = GridSearchCV(model_pipe, cv=5, param_grid={}, n_jobs=-1)
    grid_model.fit(X_train, Y_train)

    # 6. 평가
    best_model = grid_model.best_estimator_

    Y_train_pred = best_model.predict(X_train)
    Y_test_pred = best_model.predict(X_test)

    # 평가

    print(f'depth 값 : {i}')
    print(classification_report(Y_train, Y_train_pred))

    print(classification_report(Y_test,Y_test_pred))
    rf_model[f'{i}_depth'] = accuracy_score(Y_test,Y_test_pred)
rf_model

depth 값 : 5


C:\Users\DMC CONET\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DMC CONET\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DMC CONET\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

              precision    recall  f1-score   support

          A1       1.00      0.05      0.10       496
          A2       0.85      0.14      0.24      2555
          A3       0.52      0.19      0.28      6260
          B1       0.58      0.17      0.26      8240
          B2       0.36      0.83      0.50     11600
          B3       0.38      0.44      0.41      7163
          C1       0.53      0.18      0.26      4456
          C2       0.46      0.22      0.30      4298
          C3       0.52      0.67      0.58      4293
           D       0.00      0.00      0.00        11

    accuracy                           0.41     49372
   macro avg       0.52      0.29      0.29     49372
weighted avg       0.49      0.41      0.37     49372



C:\Users\DMC CONET\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DMC CONET\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\DMC CONET\AppData\Roaming\Python\Python39\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, le

              precision    recall  f1-score   support

          A1       1.00      0.05      0.10       331
          A2       0.83      0.15      0.25      1703
          A3       0.50      0.18      0.26      4173
          B1       0.59      0.17      0.26      5494
          B2       0.35      0.83      0.50      7734
          B3       0.38      0.42      0.40      4776
          C1       0.53      0.17      0.25      2971
          C2       0.43      0.21      0.29      2865
          C3       0.52      0.66      0.58      2861
           D       0.00      0.00      0.00         7

    accuracy                           0.41     32915
   macro avg       0.51      0.28      0.29     32915
weighted avg       0.48      0.41      0.36     32915

depth 값 : 6


...

              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       331
          A2       1.00      1.00      1.00      1703
          A3       1.00      1.00      1.00      4173
          B1       1.00      1.00      1.00      5494
          B2       1.00      1.00      1.00      7734
          B3       1.00      1.00      1.00      4776
          C1       1.00      1.00      1.00      2971
          C2       1.00      1.00      1.00      2865
          C3       1.00      1.00      1.00      2861
           D       1.00      1.00      1.00         7

    accuracy                           1.00     32915
   macro avg       1.00      1.00      1.00     32915
weighted avg       1.00      1.00      1.00     32915

depth 값 : 32
              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       496
          A2       1.00      1.00      1.00      2555
          A3       1.00      1.00      1.00      6260
          B

depth 값 : 37
              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       496
          A2       1.00      1.00      1.00      2555
          A3       1.00      1.00      1.00      6260
          B1       1.00      1.00      1.00      8240
          B2       1.00      1.00      1.00     11600
          B3       1.00      1.00      1.00      7163
          C1       1.00      1.00      1.00      4456
          C2       1.00      1.00      1.00      4298
          C3       1.00      1.00      1.00      4293
           D       1.00      1.00      1.00        11

    accuracy                           1.00     49372
   macro avg       1.00      1.00      1.00     49372
weighted avg       1.00      1.00      1.00     49372

              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       331
          A2       1.00      1.00      1.00      1703
          A3       1.00      1.00      1.00      4173
          B

              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       331
          A2       1.00      1.00      1.00      1703
          A3       1.00      1.00      1.00      4173
          B1       1.00      1.00      1.00      5494
          B2       1.00      1.00      1.00      7734
          B3       1.00      1.00      1.00      4776
          C1       1.00      1.00      1.00      2971
          C2       1.00      1.00      1.00      2865
          C3       1.00      1.00      1.00      2861
           D       1.00      1.00      1.00         7

    accuracy                           1.00     32915
   macro avg       1.00      1.00      1.00     32915
weighted avg       1.00      1.00      1.00     32915

depth 값 : 43
              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       496
          A2       1.00      1.00      1.00      2555
          A3       1.00      1.00      1.00      6260
          B

depth 값 : 48
              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       496
          A2       1.00      1.00      1.00      2555
          A3       1.00      1.00      1.00      6260
          B1       1.00      1.00      1.00      8240
          B2       1.00      1.00      1.00     11600
          B3       1.00      1.00      1.00      7163
          C1       1.00      1.00      1.00      4456
          C2       1.00      1.00      1.00      4298
          C3       1.00      1.00      1.00      4293
           D       1.00      1.00      1.00        11

    accuracy                           1.00     49372
   macro avg       1.00      1.00      1.00     49372
weighted avg       1.00      1.00      1.00     49372

              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       331
          A2       1.00      1.00      1.00      1703
          A3       1.00      1.00      1.00      4173
          B

{'5_depth': 0.40641045116208413,
 '6_depth': 0.4809053622968252,
 '7_depth': 0.5954428072307458,
 '8_depth': 0.712714567826219,
 '9_depth': 0.8200820294698465,
 '10_depth': 0.9171198541698314,
 '11_depth': 0.9704390095701048,
 '12_depth': 0.9937718365486861,
 '13_depth': 0.9992708491569193,
 '14_depth': 1.0,
 '15_depth': 1.0,
 '16_depth': 1.0,
 '17_depth': 1.0,
 '18_depth': 1.0,
 '19_depth': 1.0,
 '20_depth': 1.0,
 '21_depth': 1.0,
 '22_depth': 1.0,
 '23_depth': 1.0,
 '24_depth': 1.0,
 '25_depth': 1.0,
 '26_depth': 1.0,
 '27_depth': 1.0,
 '28_depth': 1.0,
 '29_depth': 1.0,
 '30_depth': 1.0,
 '31_depth': 1.0,
 '32_depth': 1.0,
 '33_depth': 1.0,
 '34_depth': 1.0,
 '35_depth': 1.0,
 '36_depth': 1.0,
 '37_depth': 1.0,
 '38_depth': 1.0,
 '39_depth': 1.0,
 '40_depth': 1.0,
 '41_depth': 1.0,
 '42_depth': 1.0,
 '43_depth': 1.0,
 '44_depth': 1.0,
 '45_depth': 1.0,
 '46_depth': 1.0,
 '47_depth': 1.0,
 '48_depth': 1.0,
 '49_depth': 1.0}

In [100]:
accuracy_score(Y_train, Y_train_pred)

1.0

In [101]:
knn_model = dict({})
for i in range(5, 50):
    model_pipe = make_pipeline(preprocessing_pipe, KNeighborsClassifier(n_neighbors=1,
                            p=2,
                            metric='minkowski'))


    grid_model = GridSearchCV(model_pipe, cv=5, param_grid={}, n_jobs=-1)
    grid_model.fit(X_train, Y_train)

    # 6. 평가
    best_model = grid_model.best_estimator_

    Y_train_pred = best_model.predict(X_train)
    Y_test_pred = best_model.predict(X_test)

    print(f'Y_train_pred : {Y_train_pred}\n')
    print(f'Y_test_pred : {Y_test_pred}\n')
    # 평가

    print(f'depth 값 : {i}')
    print(classification_report(Y_train, Y_train_pred))

    print(classification_report(Y_test,Y_test_pred))
    knn_model[f'{i}_depth'] = accuracy_score(Y_test,Y_test_pred)
knn_model

Y_train_pred : ['B3' 'C2' 'B1' ... 'B1' 'B2' 'B2']

Y_test_pred : ['B2' 'C2' 'C2' ... 'B1' 'B2' 'B2']

depth 값 : 5
              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       496
          A2       1.00      1.00      1.00      2555
          A3       1.00      1.00      1.00      6260
          B1       1.00      1.00      1.00      8240
          B2       1.00      1.00      1.00     11600
          B3       1.00      1.00      1.00      7163
          C1       1.00      1.00      1.00      4456
          C2       1.00      1.00      1.00      4298
          C3       1.00      1.00      1.00      4293
           D       1.00      1.00      1.00        11

    accuracy                           1.00     49372
   macro avg       1.00      1.00      1.00     49372
weighted avg       1.00      1.00      1.00     49372

              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       331
          A2     

              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       496
          A2       1.00      1.00      1.00      2555
          A3       1.00      1.00      1.00      6260
          B1       1.00      1.00      1.00      8240
          B2       1.00      1.00      1.00     11600
          B3       1.00      1.00      1.00      7163
          C1       1.00      1.00      1.00      4456
          C2       1.00      1.00      1.00      4298
          C3       1.00      1.00      1.00      4293
           D       1.00      1.00      1.00        11

    accuracy                           1.00     49372
   macro avg       1.00      1.00      1.00     49372
weighted avg       1.00      1.00      1.00     49372

              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       331
          A2       1.00      1.00      1.00      1703
          A3       1.00      1.00      1.00      4173
          B1       1.00 

              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       331
          A2       1.00      1.00      1.00      1703
          A3       1.00      1.00      1.00      4173
          B1       1.00      1.00      1.00      5494
          B2       1.00      1.00      1.00      7734
          B3       1.00      1.00      1.00      4776
          C1       1.00      1.00      1.00      2971
          C2       1.00      1.00      1.00      2865
          C3       1.00      1.00      1.00      2861
           D       1.00      1.00      1.00         7

    accuracy                           1.00     32915
   macro avg       1.00      1.00      1.00     32915
weighted avg       1.00      1.00      1.00     32915

Y_train_pred : ['B3' 'C2' 'B1' ... 'B1' 'B2' 'B2']

Y_test_pred : ['B2' 'C2' 'C2' ... 'B1' 'B2' 'B2']

depth 값 : 16
              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       496
          A2    

Y_train_pred : ['B3' 'C2' 'B1' ... 'B1' 'B2' 'B2']

Y_test_pred : ['B2' 'C2' 'C2' ... 'B1' 'B2' 'B2']

depth 값 : 21
              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       496
          A2       1.00      1.00      1.00      2555
          A3       1.00      1.00      1.00      6260
          B1       1.00      1.00      1.00      8240
          B2       1.00      1.00      1.00     11600
          B3       1.00      1.00      1.00      7163
          C1       1.00      1.00      1.00      4456
          C2       1.00      1.00      1.00      4298
          C3       1.00      1.00      1.00      4293
           D       1.00      1.00      1.00        11

    accuracy                           1.00     49372
   macro avg       1.00      1.00      1.00     49372
weighted avg       1.00      1.00      1.00     49372

              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       331
          A2    

              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       496
          A2       1.00      1.00      1.00      2555
          A3       1.00      1.00      1.00      6260
          B1       1.00      1.00      1.00      8240
          B2       1.00      1.00      1.00     11600
          B3       1.00      1.00      1.00      7163
          C1       1.00      1.00      1.00      4456
          C2       1.00      1.00      1.00      4298
          C3       1.00      1.00      1.00      4293
           D       1.00      1.00      1.00        11

    accuracy                           1.00     49372
   macro avg       1.00      1.00      1.00     49372
weighted avg       1.00      1.00      1.00     49372

              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       331
          A2       1.00      1.00      1.00      1703
          A3       1.00      1.00      1.00      4173
          B1       1.00 

              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       331
          A2       1.00      1.00      1.00      1703
          A3       1.00      1.00      1.00      4173
          B1       1.00      1.00      1.00      5494
          B2       1.00      1.00      1.00      7734
          B3       1.00      1.00      1.00      4776
          C1       1.00      1.00      1.00      2971
          C2       1.00      1.00      1.00      2865
          C3       1.00      1.00      1.00      2861
           D       1.00      1.00      1.00         7

    accuracy                           1.00     32915
   macro avg       1.00      1.00      1.00     32915
weighted avg       1.00      1.00      1.00     32915

Y_train_pred : ['B3' 'C2' 'B1' ... 'B1' 'B2' 'B2']

Y_test_pred : ['B2' 'C2' 'C2' ... 'B1' 'B2' 'B2']

depth 값 : 32
              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       496
          A2    

Y_train_pred : ['B3' 'C2' 'B1' ... 'B1' 'B2' 'B2']

Y_test_pred : ['B2' 'C2' 'C2' ... 'B1' 'B2' 'B2']

depth 값 : 37
              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       496
          A2       1.00      1.00      1.00      2555
          A3       1.00      1.00      1.00      6260
          B1       1.00      1.00      1.00      8240
          B2       1.00      1.00      1.00     11600
          B3       1.00      1.00      1.00      7163
          C1       1.00      1.00      1.00      4456
          C2       1.00      1.00      1.00      4298
          C3       1.00      1.00      1.00      4293
           D       1.00      1.00      1.00        11

    accuracy                           1.00     49372
   macro avg       1.00      1.00      1.00     49372
weighted avg       1.00      1.00      1.00     49372

              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       331
          A2    

              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       496
          A2       1.00      1.00      1.00      2555
          A3       1.00      1.00      1.00      6260
          B1       1.00      1.00      1.00      8240
          B2       1.00      1.00      1.00     11600
          B3       1.00      1.00      1.00      7163
          C1       1.00      1.00      1.00      4456
          C2       1.00      1.00      1.00      4298
          C3       1.00      1.00      1.00      4293
           D       1.00      1.00      1.00        11

    accuracy                           1.00     49372
   macro avg       1.00      1.00      1.00     49372
weighted avg       1.00      1.00      1.00     49372

              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       331
          A2       1.00      1.00      1.00      1703
          A3       1.00      1.00      1.00      4173
          B1       1.00 

              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       331
          A2       1.00      1.00      1.00      1703
          A3       1.00      1.00      1.00      4173
          B1       1.00      1.00      1.00      5494
          B2       1.00      1.00      1.00      7734
          B3       1.00      1.00      1.00      4776
          C1       1.00      1.00      1.00      2971
          C2       1.00      1.00      1.00      2865
          C3       1.00      1.00      1.00      2861
           D       1.00      1.00      1.00         7

    accuracy                           1.00     32915
   macro avg       1.00      1.00      1.00     32915
weighted avg       1.00      1.00      1.00     32915

Y_train_pred : ['B3' 'C2' 'B1' ... 'B1' 'B2' 'B2']

Y_test_pred : ['B2' 'C2' 'C2' ... 'B1' 'B2' 'B2']

depth 값 : 48
              precision    recall  f1-score   support

          A1       1.00      1.00      1.00       496
          A2    

{'5_depth': 0.9991493240164059,
 '6_depth': 0.9991493240164059,
 '7_depth': 0.9991493240164059,
 '8_depth': 0.9991493240164059,
 '9_depth': 0.9991493240164059,
 '10_depth': 0.9991493240164059,
 '11_depth': 0.9991493240164059,
 '12_depth': 0.9991493240164059,
 '13_depth': 0.9991493240164059,
 '14_depth': 0.9991493240164059,
 '15_depth': 0.9991493240164059,
 '16_depth': 0.9991493240164059,
 '17_depth': 0.9991493240164059,
 '18_depth': 0.9991493240164059,
 '19_depth': 0.9991493240164059,
 '20_depth': 0.9991493240164059,
 '21_depth': 0.9991493240164059,
 '22_depth': 0.9991493240164059,
 '23_depth': 0.9991493240164059,
 '24_depth': 0.9991493240164059,
 '25_depth': 0.9991493240164059,
 '26_depth': 0.9991493240164059,
 '27_depth': 0.9991493240164059,
 '28_depth': 0.9991493240164059,
 '29_depth': 0.9991493240164059,
 '30_depth': 0.9991493240164059,
 '31_depth': 0.9991493240164059,
 '32_depth': 0.9991493240164059,
 '33_depth': 0.9991493240164059,
 '34_depth': 0.9991493240164059,
 '35_depth': 0.

In [102]:
log_model = LogisticRegression()
rnd_model = RandomForestClassifier()
xgb_model = XGBClassifier
svm_model = SVC()

# 앙상블 모델 구축
# 만약에 모든 모델이 predict_proba() 메서드가 있으면, 예측의 평균을 내어 soft voting(간접 투표)도 할수 있다.
# 간접 투표 방식은 확률이 높은 투표에 비중을 두기 때문에 성능이 더 높다. (voting='soft' 사용)
# svc는 기본적으로 predict_proba를 제공하지 않아, probability = True 지정 해야 사용 가능
# 대신 svc에서 probability = True를 지정하면 교차 검증을 사용해서 확률을 추정하기 때문에 훈련 속도 느려짐
# 대신 성능을 올라감
ss = 중분류별_col.copy()
ss = ss.dropna()

X = ss[[ '당좌자산', '매출액', '부채와자본총계', '영업이익', '유동부채', '유동자산', '이익잉여금','자본잉여금','이자비용',  '자산총계',
       '조사년도', '종업원수', # '재무제표기준년월', '중분류항목명_x', '기업규모명' ,
       '유동비율', '이자보상비율', '매출영업이익률', '총자산회전율', '당좌비율']] 

Y = ss['CRI등급']

voting_model = VotingClassifier(
    estimators=[('lr',log_model),('rf',rnd_model),('svc',svm_model)], # 3개의 약한 학습기
    voting='hard' # 직접 투표(hard voting)
)

# 앙상블 모델 학습
voting_model.fit(X_train,Y_train)

# 모델 비교
for model in (log_model,rnd_model,svm_model,voting_model):
    model.fit(X_train,Y_train)
    y_pred = model.predict(X_test)
    print(model.__class__.__name__," : ",accuracy_score(y_test,Y_pred))


KeyError: "['유동비율', '이자보상비율', '매출영업이익률', '총자산회전율', '당좌비율'] not in index"

# 소분류

In [ ]:
input_ff['소분류항목명'].value_counts().index.tolist()

In [ ]:
input_ff.pivot_table(index='소분류항목명', columns='재무제표계정과목명',values='재무제표계정과목금액')

In [ ]:
# index=['소분류항목명', '사업자등록번호'], columns=['재무제표계정과목명'], values='재무제표계정과목금액'
input_ff_cl = input_ff.pivot_table(index=['소분류항목명', '사업자등록번호'], columns=['재무제표계정과목명'], values='재무제표계정과목금액')
input_ff_cl.reset_index()

In [ ]:
input_ff_cl = input_ff.pivot_table(index=['사업자등록번호','소분류항목명'], columns=['재무제표계정과목명'], values='재무제표계정과목금액')
input_ff_cl = input_ff_cl.reset_index() #['당좌자산', '매출액', '부채와자본총계', '영업이익', '유동부채', '유동자산', '이익잉여금', '이자비용','자본잉여금', '자산총계']
소분류_col = input_ff_cl.merge(input_ff, right_on='사업자등록번호', left_on='사업자등록번호', how='left')[['소분류항목명_x', '당좌자산', '매출액', '영업이익', '유동부채', '유동자산',
       '이익잉여금', '이자비용', '자본잉여금', '자산총계', '조사년도', '재무제표기준년월',  '기업규모명', 'CRI등급',  '종업원수', ]]

In [ ]:
import numpy as np
# ff = ff.replace([np.inf, -np.inf], 1)

import numpy as np

소분류_col = 소분류_col.replace([np.inf, -np.inf], 1)
소분류_col = 소분류_col.fillna(소분류_col.mean())

X = 소분류_col[['소분류항목명_x',#, '당좌자산', '매출액', '부채와자본총계', '영업이익', '유동부채', '유동자산', '이익잉여금','자본잉여금','이자비용',  '자산총계',
        '조사년도', #'재무제표기준년월','기업규모명', '종업원수',
       '유동비율', '이자보상비율', '매출영업이익률', '총자산회전율', '당좌비율']] 
       
Y = 소분류_col['CRI등급']

In [ ]:
X = 소분류_col[['소분류항목명_x',#, '당좌자산', '매출액', '부채와자본총계', '영업이익', '유동부채', '유동자산', '이익잉여금','자본잉여금','이자비용',  '자산총계',
        '조사년도', #'재무제표기준년월','기업규모명', '종업원수',
       '유동비율', '이자보상비율', '매출영업이익률', '총자산회전율', '당좌비율']] 
       
Y = 소분류_col['CRI등급']